<a href="https://colab.research.google.com/github/rya23/llm-from-scratch/blob/master/first_ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Load the Dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-06 13:22:46--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-08-06 13:22:46 (17.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [11]:
with open('input.txt' , 'r',encoding ='utf-8') as f:
  text = f.read()

In [12]:
text[:100]
len(text)

1115394

In [44]:
chars = sorted(list(set(text)))
print(''.join(chars))
vocabulary_size = len(chars)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [23]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s : [stoi[c] for c in s]
decode = lambda s : ''.join([itos[c] for  c in s])

In [27]:
print(encode("hello there"))
print(decode(encode("hello there")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [29]:
import torch

In [30]:
data = torch.tensor(encode(text),dtype=torch.long)
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [34]:
#train test split
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

torch.Size([1115394])

In [42]:
block_size = 8 # size of context to be trained
batch_size = 4 # Independent Parallel data blocks to be loaded at once


def generate_batch(split):

    data = train_data if split == 'train' else test_data
    random_points = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_points])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_points])

    return x,y

In [43]:
generate_batch('abcd')

(tensor([[56, 39, 63,  1, 63, 53, 59,  6],
         [39, 52, 42,  1, 45, 47, 60, 43],
         [39, 50, 50,  1, 52, 43, 60, 43],
         [51, 63,  1, 63, 53, 59, 52, 45]]),
 tensor([[39, 63,  1, 63, 53, 59,  6,  1],
         [52, 42,  1, 45, 47, 60, 43,  1],
         [50, 50,  1, 52, 43, 60, 43, 56],
         [63,  1, 63, 53, 59, 52, 45, 43]]))

##Logits are non normalized predictions which are fed to the softmax function